#### 1) Запросить курсы любой валюты за последний год. Сохранить полученные данные в базу данных Mongo.  
#### 2) Написать функцию, которая принимает в качестве параметров две даты, ищет самую большую разницу между курсами валюты за указанный период и возвращает эти дни.  
#### 3) Сделать вывод информативным: Валюту <Валюта> выгодно было купить <Дата>, а продать <Дата>. Прибыль: <Разница в цене валюты>

In [1]:
import zeep
import pprint
import json
import datetime
from pymongo import MongoClient
import time

In [2]:
url = "https://www.cbr.ru/DailyInfoWebServ/DailyInfo.asmx?WSDL"
zeepclient = zeep.Client(url)

Forcing soap:address location to HTTPS
Forcing soap:address location to HTTPS


In [3]:
mongoclient = MongoClient('mongodb://127.0.0.1:27017')
db = mongoclient['currencies']
currdb = db.currencies

In [4]:
def date_to_seconds(date):
    """
    Date example:
        date = '2018-12-30'
    """
    return int(datetime.datetime.timestamp(datetime.datetime.fromisoformat(date)))

def seconds_to_date(seconds):
    """
    Seconds example:
        seconds = 1576083600.0
    """
    return datetime.datetime.fromtimestamp(seconds).strftime('%Y-%m-%d')

def date_to_datetime(date):
    return datetime.datetime.fromisoformat(date)

def datetime_to_date(datetime_val):
    return datetime_val.strftime('%Y-%m-%d')

def datetime_to_seconds(datetime_val):
    return int(datetime.datetime.timestamp(datetime_val))

def seconds_to_datetime(seconds):
    return date_to_datetime(seconds_to_date(seconds))

In [5]:
def date_curs(datetime_val):
    date = datetime_to_date(datetime_val)
    money = zeepclient.service.GetCursOnDate(date)
    new_valute_dict = {}
    for rec in money._value_1._value_1:
        key = rec["ValuteCursOnDate"]["VchCode"]
        value = {
            "Vnom": float(rec["ValuteCursOnDate"]["Vnom"]),
            "Vcurs": {str(datetime_to_seconds(datetime_val)): round(float(rec["ValuteCursOnDate"]["Vcurs"]), 4)},
            "Vname": rec["ValuteCursOnDate"]["Vname"].rstrip()
        }
        new_valute_dict.update({key: value})
    return new_valute_dict

In [6]:
def show_what_in_database(print_limit=0):
    cursor = currdb.find()
    i = 0
    for document in cursor:
        pprint.pprint(document)
        print()
        i += 1
        if print_limit != 0:
            if i == print_limit:
                return

In [7]:
# Начальная дата, с которой будут добавлены все курсы валют (Формат: '2016-12-31')
START_DATE = "2009-12-31"

In [8]:
start_datetime = date_to_datetime(START_DATE)
today_datetime = datetime.datetime.now()
print("Сегодня:", datetime_to_date(today_datetime))

Сегодня: 2019-07-11


In [9]:
# Drop database:
# mongoclient.drop_database("currencies")

In [10]:
start_time = time.time()
tmp_datetime = start_datetime
print_step = 365
while (today_datetime - tmp_datetime).days >= 0:
    tmp_date = datetime_to_date(tmp_datetime)
    tmp_curs = date_curs(tmp_datetime)
    if (tmp_datetime - start_datetime).days % print_step == 0 and tmp_datetime != start_datetime:
        print("%d дней до" % print_step, tmp_date, "обработано за %.4f секунд" % (time.time() - start_time))
        start_time = time.time()
    for valute in tmp_curs:
        if currdb.count_documents({valute: { "$exists": True}}) == 0:
            currdb.insert_one({valute: tmp_curs[valute]})
        else:
            vcurs_dict = currdb.find_one({valute: { "$exists": True}})[valute]['Vcurs']
            vnom = currdb.find_one({valute: { "$exists": True}})[valute]['Vnom']
#             Номинал иногда меняется, поэтому надо его выравнивать
            if vnom != tmp_curs[valute]['Vnom']:
                tmp_multiplier = tmp_curs[valute]['Vnom'] / vnom
                tmp_curs[valute]['Vcurs'][str(datetime_to_seconds(tmp_datetime))] = round(tmp_curs[valute]['Vcurs'][str(datetime_to_seconds(tmp_datetime))] / tmp_multiplier, 4)
            vcurs_dict.update({str(datetime_to_seconds(tmp_datetime)): tmp_curs[valute]['Vcurs'][str(datetime_to_seconds(tmp_datetime))]})
            
            currdb.update_one({valute: { "$exists": True}}, 
                              {'$set': {'%s.Vcurs' % valute: vcurs_dict}}, upsert=True)
    tmp_datetime += datetime.timedelta(days=1)

365 дней до 2010-12-31 обработано за 81.0376 секунд
365 дней до 2011-12-31 обработано за 83.7347 секунд
365 дней до 2012-12-30 обработано за 79.9743 секунд
365 дней до 2013-12-30 обработано за 93.7467 секунд
365 дней до 2014-12-30 обработано за 109.7251 секунд
365 дней до 2015-12-30 обработано за 98.5174 секунд
365 дней до 2016-12-29 обработано за 107.7295 секунд
365 дней до 2017-12-29 обработано за 110.7166 секунд
365 дней до 2018-12-29 обработано за 109.0210 секунд


### Доступные валюты в базе:

In [11]:
print("Доступные валюты:")
for doc in currdb.find():
    for key in doc:
        if key != "_id":
            print(key, "-", doc[key]["Vname"])

Доступные валюты:
AUD - Австралийский доллар
GBP - Фунт стерлингов Соединенного королевства
BYR - Белорусский рубль
DKK - Датская крона
USD - Доллар США
EUR - Евро
ISK - Исландская крона
KZT - Казахстанский тенге
CAD - Канадский доллар
CNY - Китайский юань
NOK - Норвежская крона
XDR - СДР (специальные права заимствования)
SGD - Сингапурский доллар
TRY - Турецкая лира
UAH - Украинская гривна
SEK - Шведская крона
CHF - Швейцарский франк
JPY - Японская иена
AZN - Азербайджанский манат
AMD - Армянский драм
BGN - Болгарский лев
BRL - Бразильский реал
HUF - Венгерский форинт
INR - Индийская рупия
KGS - Киргизский сом
LVL - Латвийский лат
LTL - Литовский лит
MDL - Молдавский лей
PLN - Польский злотый
RON - Румынский лей
TJS - Таджикский сомони
TMT - Новый туркменский манат
UZS - Узбекский сум
CZK - Чешская крона
EEK - Эстонская крона
ZAR - Южноафриканский рэнд
KRW - Вон Республики Корея
BYN - Белорусский рубль
HKD - Гонконгский доллар


In [12]:
def find_good_purchase(from_date="1990-12-31", to_date="2020-12-31", currency="USD"):
    currency_data = currdb.find_one({currency: { "$exists": True}})
    from_seconds = date_to_seconds(from_date)
    to_seconds = date_to_seconds(to_date)
    vcurs_keys = [int(key) for key in currency_data[currency]["Vcurs"]]
    
    from_index = -1
    to_index = len(vcurs_keys) - 1
    
    if from_seconds < vcurs_keys[0]:
        print("Вы хотели данные начиная с %s," % from_date, "но для %s данные доступны начиная с %s." % (currency, seconds_to_date(vcurs_keys[0])))
        return
    
    if to_seconds > vcurs_keys[-1]:
        print("Вы хотели данные заканчивая %s," % to_date, "но для %s данные доступны только до %s" % (currency, seconds_to_date(vcurs_keys[-1])))
        return
    
    for i in range(len(vcurs_keys)):
        if from_index == -1 and vcurs_keys[i] >= from_seconds:
            from_index = i
        if vcurs_keys[i] <= to_seconds:
            to_index = i
    
    max_diff_start_idx = from_index
    max_diff_end_idx = from_index
    max_diff = -9999999.99
    for left_idx in range(from_index, to_index):
        for righ_idx in range(left_idx, to_index+1):
            tmp_diff = currency_data[currency]['Vcurs'][str(vcurs_keys[righ_idx])] - currency_data[currency]['Vcurs'][str(vcurs_keys[left_idx])]
            if tmp_diff > max_diff:
                max_diff_start_idx = left_idx
                max_diff_end_idx = righ_idx
                max_diff = tmp_diff
    
    print("Валюту", currency, "выгодно было купить %s," % seconds_to_date(vcurs_keys[max_diff_start_idx]), "а продать %s." % seconds_to_date(vcurs_keys[max_diff_end_idx]), "Прибыль: %.4f руб. с каждых %d проданных %s" % (max_diff, currency_data[currency]['Vnom'], currency)) 

In [13]:
find_good_purchase()

Вы хотели данные начиная с 1990-12-31, но для USD данные доступны начиная с 2009-12-31.


In [14]:
find_good_purchase(from_date="2012-01-01")

Вы хотели данные заканчивая 2020-12-31, но для USD данные доступны только до 2019-07-11


In [15]:
find_good_purchase(from_date="2015-12-31")

Вы хотели данные заканчивая 2020-12-31, но для USD данные доступны только до 2019-07-11


In [16]:
find_good_purchase(from_date="2016-01-03", to_date="2019-01-26")

Валюту USD выгодно было купить 2018-02-28, а продать 2018-09-12. Прибыль: 14.3027 руб. с каждых 1 проданных USD


In [20]:
find_good_purchase(from_date="2010-01-01", to_date="2019-01-26")

Валюту USD выгодно было купить 2011-05-06, а продать 2016-01-22. Прибыль: 56.3288 руб. с каждых 1 проданных USD


In [17]:
find_good_purchase(from_date="2016-06-03", to_date="2019-01-26", currency="HKD")

Вы хотели данные начиная с 2016-06-03, но для HKD данные доступны начиная с 2017-04-01.


In [18]:
find_good_purchase(from_date="2017-06-03", to_date="2019-01-26", currency="HKD")

Валюту HKD выгодно было купить 2018-02-28, а продать 2018-09-12. Прибыль: 18.0073 руб. с каждых 10 проданных HKD


In [19]:
find_good_purchase(from_date="2019-06-05", to_date="2019-01-26", currency="HKD")

Валюту HKD выгодно было купить 2019-06-05, а продать 2019-06-05. Прибыль: -9999999.9900 руб. с каждых 10 проданных HKD


In [28]:
find_good_purchase(from_date="2010-01-12", to_date="2019-01-26", currency="INR")

Валюту INR выгодно было купить 2013-08-29, а продать 2016-01-22. Прибыль: 74.2486 руб. с каждых 100 проданных INR


In [29]:
find_good_purchase(from_date="2010-01-12", to_date="2019-01-26", currency="HUF")

Валюту HUF выгодно было купить 2012-01-18, а продать 2016-01-22. Прибыль: 16.0019 руб. с каждых 100 проданных HUF


### Уже после того как закончил (и озадачился долгим выполнением запросов к ЦБРФ при заполнении базы данными) нашел, что можно было через другой запрос получать для валюты курс в заданном периоде. Переделывать уже не стал. С другой стороны та реализация которая есть на текущий момент имеет право на жизнь, так как добавление архивной информации в базу происходит единожды, а дальнейшие дополнения в базу уже будут для более свежих дат небольшими интервалами, и, как следствие, быстро.

In [26]:
# zeepclient.service.GetCursDynamic(FromDate, ToDate, ValutaCode)
# ValutaCode смотреть здесь: http://www.cbr.ru/scripts/XML_valFull.asp
USD_ValuteCode = "R01235"
zeepclient.service.GetCursDynamic("2019-06-25", "2019-07-11", USD_ValuteCode)

{
    'schema': <Schema(location=None, tns=None)>,
    '_value_1': {
        '_value_1': [
            {
                'ValuteCursDynamic': {
                    'CursDate': datetime.datetime(2019, 6, 25, 0, 0, tzinfo=<FixedOffset '+03:00'>),
                    'Vcode': 'R01235',
                    'Vnom': Decimal('1'),
                    'Vcurs': Decimal('62.9095')
                }
            },
            {
                'ValuteCursDynamic': {
                    'CursDate': datetime.datetime(2019, 6, 26, 0, 0, tzinfo=<FixedOffset '+03:00'>),
                    'Vcode': 'R01235',
                    'Vnom': Decimal('1'),
                    'Vcurs': Decimal('62.5229')
                }
            },
            {
                'ValuteCursDynamic': {
                    'CursDate': datetime.datetime(2019, 6, 27, 0, 0, tzinfo=<FixedOffset '+03:00'>),
                    'Vcode': 'R01235',
                    'Vnom': Decimal('1'),
                    'Vcurs': Decimal('62.8